# RareSkills Zero Knowledge Week 2


## Modular arithmetic essentials
### Benchmarks
Here is how you do modular arithmetic in python. Run the two cells below and note how different their execution times are. You should use `pow` instead of doing modular arithmetic with the elementary operators.

In [ ]:
import time

s = time.time()
ans = 1002583 ** 939001 % 2003951
t = time.time()

print("calculating {} took {:.2f} seconds".format(ans, t - s))

In [ ]:
s = time.time()
ans = pow(1002583,939001,2003951)
t = time.time()

print("calculating {} operation took {:.2f} seconds".format(ans, t - s))

### Encrypting a number

If we start with a generator `g` and raise it to a power `s` mod `n`, then in general we cannot determine `s` even if `g` and `n` are known. This is the discrete logarithm problem.

We can say we "encrypt" `s` using the scheme

$$ E(s) = g^{s} \pmod n $$

But this means we need to choose `g` and `n` carefully. If `g` is 3, `s` is 2, and `n` is a large number, we can intelligently guess what `s` is in this case. We want to pick `g` and `n` such that all members "wrap around." We won't concern ourselves with this here, but you should be aware of it. The other issue is not picking a sufficiently large `n`. 

### Computing the discrete logarithm
If we do `s_p = g^s mod n`, and make `g` and `n` public, then it is infeasible to compute `s`. But this is only true if `n` is sufficiently large.

In the following example, we provide `s_p`, `g`, and `n`. Use python to bruteforce the solution to find `s`.

$$g^{ac} = (g^{a})^{c} \pmod p$$

In [2]:
n = 9551
g = 5
encrypted_number = 5666

<font color='red'>**Assignment 1: Use python to bruteforce the solution to find `s`**</font>

In [3]:
# Your Solution and Answer Here
student_solution = 0
for i in range(n):
    if pow(g, i, n) == 5666:
        student_solution = i
        break
print(student_solution)

2531


In [4]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 2531


### Computing the discrete logarithm the smart way
The smart way to do optimal algorithms is to [copy and paste from stackoverflow](https://stackoverflow.com/a/58665206/2079806). You do not have to understand how the library works

<font color='red'>**Assignment 2: Break the cryptography below and obtain the secret number**</font>

In [5]:
n = 1000004119
g = 5
encrypted_number = 767805982

In [7]:
# Your Solution and Answer Here
from sympy.ntheory import discrete_log

student_solution = discrete_log(n,encrypted_number , g)

In [8]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 420


## How Zero Knowledge Addition Works

A Zero Knowledge Proof verifies a computation is true without revealing a the inputs to the computation.

That is, if a + b = c, then we want a "trapdoor function" E(x) such that E(a)■E(b) = E(c) where ■ is the binary operator of the group E(x) is an element of.

So to prove a + b = c without revealing a, b, or c, we apply A = E(a), B = E(b), C = E(c) and give (A, B, C) to a verifier.

The verifier will then check A ■ B == C.

Although they don't know the values A, B, and C "came from", they do know the unencrypted values form a valid sum.

## Zero Knowledge Addition
The following property is very important. It lets us verify the addition of numbers (integers under addition) using another group: $g^x \pmod p$ under multiplication. Because of the discrete logarithm, we cannot easily go from $g^x \pmod p$ to recover x.

a + b = c is homomorphic to $g^ag^b = g^c \pmod p$.

$$ g^{a}g^{b} = g^{a + b} \pmod n $$

Try a few different values of `a` and `b` yourself to see it in action

In [10]:
a = 214031242144124012 % n
b = 8094293523502359235 % n
print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n))

False


Wait what?! That's supposed to be true?

<font color='red'>**Assignment 3: The code above has a bug. What is the bug?**</font>

In [18]:
# Your corrected code here
# Don't forget (mod n)
print((pow(g, a, n) * pow(g, b, n))%n == pow(g, a + b, n) % n)

True


## Zero Knowledge Subtraction
We can also encrypt the operation a - b. This is the same as 

$$ g^{a}g^{-b} = g^{a-b} $$

But we can't just stick a negative sign in front of the exponent

In [25]:
g = 57
n = 101
g ** -5 % n

1.6619797259514097e-09

The above is not an integer! However, python is smart enough to know what you are doing if you use `pow`. To test this, we expect

$$ g^{-a}g^{a} = 1 $$

because g^0 is 1, and `a - a = 0`.

In [31]:
a = 22
pow(g, a, n) * pow(g, -a, n) % n == 1

True

So what magic is happening behind the scenes? The formula below is used to compute the modular inverse.

$$ a^{-1} = a^{n - 2} \pmod n $$

In [36]:
a_inv = a ** (n - 2) % n
a_inv * a % n == 1

True

## Multiplication by a constant
Multiplication by a constant is really just repeated addition

$$ (g^{a})^{4} = g^{a} g^{a} g^{a} g^{a} $$

This allows us to "multiply by a constant" in a zero knowledge fashion.

In [40]:
a = 15
pow(g, a, n) * pow(g, a, n) * pow(g, a, n) * pow(g, a, n) % n == pow(g, a * 4, n)

True

Of course, it would be annoying to multiply like that if you have a big coefficient, so the following is better

In [41]:
pow(pow(g, a, n), 4 , n) == pow(g, a * 4, n)

True

## I know the solution to a systems of equations

Now you should be able to prove to me that you know the solution to in a zero knowledge fashion

$$
2x + 8y = 7944\\
5x + 3y = 4764
$$

<font color='red'>**Assignment 4: conver the above equation to a zk proof system where you can prove to a verifier (with an agreed upon g and n) that you know the solution to an agreed upon system of equation, without revealing the solution**</font>

In [11]:
"""Part 1: Solving the equation"""

import numpy as np
import galois


g = 5
n = 71

n = 37

GF = galois.GF(n)

A = GF([[2, 8], 
        [5, 3]])

B = GF([7944 % n, 4764 % n])

# Now solve the equation Av = B (v = [x, y])

[x, y] = np.linalg.solve(A, B)

# Convert to int
x = int(x)
y = int(y)

print(x, y)

# Assert that these x,y values satisfy the simultaneous equations
assert((2*x + 8*y) % n == 7944 % n)
assert((5*x + 3*y) % n == 4764 % n)

13 0


In [5]:

"""Part 2: Prover land"""
# So I want to prove that I know x, y that satisfy the equations without giving away x,y
# Well I know that the system should be homomorphic to:
# g^2x * g^8y = g^7944 (mod n)
# g^5x * g^3y = g^4764 (mod n)
# So I can provide g^x (mod n) and g^y (mod n), tell him to compute the above

# Public values to give to verifier:
gx = pow(g, x, n) # g^x (mod n)
gy = pow(g, y, n) # g^y (mod n)

"""Part 3: Verifier land"""
# This prover has given me gx and gy, but i have no clue what x or y is (discrete log problem)

# I can still check the equation in g^x (mod n) land, and know that he knows the solution

print((pow(gx, 2, n) * pow(gy, 8, n)) % n)
print(pow(g, 4764, n))
print((pow(gx, 79445, n) * pow(gy, 3, n)) % n)

assert((pow(gx, 2, n) * pow(gy, 8, n)) % n == pow(g, 4764, n))

assert((pow(gx, 79445, n) * pow(gy, 3, n)) % n == pow(g, 4764, n))

1634636351
124876100
815661965


AssertionError: 

## Convert a rational number to a finite field element

<font color='red'>**Assignment 5: Compute 53/192 + 61/511 (mod 1033) in python. Show your work. You are not required to implement euclid's algorithm. Show it is equal to the original rational number sum: https://www.wolframalpha.com/input?i=53%2F192+%2B+61%2F511**</font>

In [ ]:
p = 1033
# a = (53 * pow(192, p-2, p) + 61*pow(511, p-2, p)) % p
a  = (53 * pow(192, -1, p) + 61*pow(511, -1, p)) % p

print(a) # 514

print((38795 * pow(98112, -1, p)) % p)

assert(a == (38795 * pow(98112, -1, p)) % p)

514
514
